In [1]:
import copy
class Node(object):
    """Node类. 作用:开辟一个树节点"""
    def __init__(self, data=-1, count=0, parent=None):
        """Node类的初始化, 一个节点信息包括:项的值,计数,父亲节点,所有孩子节点"""
        self.data = data
        self.count = count
        self.parent = parent
        self.children = {}
    
    
class Tree(object):
    """tree_growth类. 作用:建造树"""
    def __init__(self, data=-1, parent=None, itemTable=None):
        """tree_growth类的初始化,开辟一个树根"""
        self.root = Node(data='null', parent=self)
        self.itemTable = itemTable
    
    
    def addRoutine(self, routine, Rroot, count):
        """功能:根据事务routine递归构造树, Rroot是树的根节点, count是routine出现的次数(构造条件FP_tree的时候游泳)"""
        if len(routine) <= 0:       #如果事务为空，则终止
            return
        elem = routine.pop(0)
        if elem in Rroot.children:          #如果事务中的元素在树的已有路径上
            nextNode = Rroot.children[elem]          #如果事务中的元素在树的已有路径上 ，则不用新建路径 
        else:                                       #否则，新建一条路径
            newNode = Node(data=elem, parent=Rroot)         #新建一个节点
            Rroot.children.setdefault(elem,newNode)         #新节点的路径放在当前节点的孩子列表中
            nextNode = newNode
        nextNode.count += count         #更新路径上节点的计数，即加上当前节点的计数
        if nextNode not in self.itemTable[elem]:        #如果下一个节点是新建的，则把它压入头结点表中
            self.itemTable[elem].append(nextNode)
        self.addRoutine(routine=routine, Rroot=nextNode, count=count)           #递归构造树
        return
class Tree_builder(object):
    """tree_builder类。 作用:根据事务数据集进行数据准备及构造树."""
    def __init__(self, routines, min_sup=-1, counts=[], headerTable={}):
        """类的初始化。 routines:事务数据集; min_sup:最小支持度及数; counts:每个事务出现的次数,默认为1; headerTable:头结点表,建造的FP_tree各结点的索引表。"""
        self.routines = routines
        self.counts = counts
        self.min_sup = min_sup
        self.items = self.getItems(self.routines)          #获取所有项
        self.sortedItems = self.getSortedItems(self.items)        #对所有项进行并排序,把计数小于min_sup的项移除，生成有序的频繁1-项集
        self.itemsTable = self.initItemsTable(headerTable)
        self.tree = self.treeBuilding(self.counts)
    
    
    def getItems(self, routines):
        """功能:扫描事务数据集,返回它的项集即各项的计数"""
        items = {}
        for routine in routines:
            for elem in routine:
                    items.setdefault(elem,0)
                    items[elem] += 1
        return items
    
    
    def getSortedItems(self, items=None):
        """功能:对项集进行排序,并删去非频繁项,得到频繁1-项集"""
        sortedItems = []
        temp = sorted(items.iteritems(), key=lambda asd:asd[1], reverse=True)       #对字典items进行排序
        for elem in temp:
            if elem[1] >= self.min_sup:             #只取计数大于等于最小支持度及数的项
                sortedItems.append(elem[0])
        return sortedItems
    
    
    def getSortedRoutine(self, routine):
        """功能:根据排序好的频繁1-项集对某一条事务routine进行排序"""
        sortedRoutine = []
        for elem in self.sortedItems:
            if elem in routine:
                sortedRoutine.append(elem)
        return sortedRoutine
    
    
    def initItemsTable(self, itemsTable):
        """功能:头结点表的初始化"""
        for item in self.sortedItems:
            itemsTable.setdefault(item,[])
        return itemsTable
    
    
    def treeBuilding(self, counts):
        """功能:逐条取出事务,控制FP_Tree树的构造"""
        tree = Tree(itemTable=self.itemsTable)             #生成一个树对象
        for routine in self.routines:                           #对事物数据集中的事务，逐条进行构造树
            sortedRoutine = self.getSortedRoutine(routine)          #用一条事务构造树之前先进行排序
            if counts:                      #如果counts不为空，即是在用模式基在构造条件树，此时需要考虑模式基的计数
                count = counts.pop(0)
            else:
                count =1
            tree.addRoutine(routine=sortedRoutine, Rroot=tree.root, count=count)             #用排序好的事务构造树
        return tree



class Tree_miner(object):
    """tree_miner类. 作用:对Tree进行频繁项集的挖掘"""
    def __init__(self, Tree=None, min_sup=-1, headerTable={}):
        """tree_miner的初始化. Tree即为构造好的FP_Tree, min_sup是最小支持度计数, headerTable是FP_Tree的头结点表"""
        self.min_sup = min_sup
        self.tree_mining(Tree=Tree, headerTable=headerTable)
    
    
    def tree_mining(self, Tree, A=[], headerTable={}):
        """功能: 递归实现对树Tree频繁项集的挖掘. A相当于伪代码中的α，B相当于β"""
        B = []
        allElem = {}        #用来保存单个路径情况时，路径上的所有节点
        node = Tree.root       #node取得树的根节点
        while len(node.children) > 0:        #判断是否是单个路径
            if len(node.children) != 1:          #如果路径上的某个节点的孩子数不止一个，则它不是单个路径
                break
            node = node.children.values()[0]        #node取得下一个节点
            allElem.setdefault(node.data,node.count)        #记录路径上的节点，如果是单个路径的话会用到
        if len(node.children) < 1:                  #Tree只包含单个路径
            L = self.getL(items=allElem, min_sup=self.min_sup, A=A)     #L即为我们要求的频繁项集
            self.showResult(L)      #对结果进行输出
            return
        else:
            for item in headerTable:            #对于头结点表中的元素，逐个找以其结尾的频繁项集
                if A:                   #产生项目集B
                    for elem in A:
                        if elem != []:
                            temp = copy.copy(elem)
                            B.append(temp)
                            B.append([item]+temp)
                else:
                    B.append([item])
                pattem,counts = self.findPattemBase(item, headerTable)      #得到以项item结尾的所以条件模式基,counts存放条件模式基的计数
                myHeaderTable = {}          
                conditionTree_builder = Tree_builder(routines=pattem, counts=counts, headerTable=myHeaderTable)        #新建一个Tree_builder对象，用它来构造条件FP-Tree
                if conditionTree_builder.tree.root.children:            #如果构造的条件FP-树不空
                    self.tree_mining(Tree=conditionTree_builder.tree, A=B, headerTable=myHeaderTable)       #递归调用
                B = []
        return
    
    
    def findPattemBase(self, item, headerTable):
        """功能: 根据树的头结点表去搜索树中item的条件模式基"""
        itemPattem = []                 #存放项item的所有模式基
        counts = []                     #存放模式基的计数
        addressTable = headerTable[item]    #头节点表中item链上所以节点的地址
        for itemNode in addressTable:           #对头结点表表中存放的每个item节点
            itemInPattem = []               #用来存放item模式基中的各项
            nodeInPattem = itemNode.parent         #item模式基的项，用它来回溯到树根，即为一条模式基
            if nodeInPattem.data == 'null':         #如果父亲节点就是树根，则跳过
                continue
            while nodeInPattem.data != 'null':                  #如果还没到树根，则一直回溯
                itemInPattem.append(nodeInPattem.data)           #把它压进item的模式基
                nodeInPattem = nodeInPattem.parent          #让当前节点跳到它的父亲节点，进行回溯
            itemInPattem = tuple(itemInPattem)
            itemPattem.append(itemInPattem)             #找完了一条item的模式基了
            counts.append(itemNode.count)           #模式基的计数
        return itemPattem,counts
    
    
    def showResult(self, result=[[]]):
        """功能: 将挖掘到的频繁项集进行展示"""
        for elem in result:
            num = elem.pop()        #频繁项集的计数
            print tuple(elem),':',num
        return
    
    
    def combiner(self, myList, n): 
        """功能: 对list列表里的所有元素进行排列组合,生成n个元组组合在一起的列表"""
        answers = []
        one = [0] * n 
        def next_c(li = 0, ni = 0): 
            if ni == n:
                answers.append(copy.copy(one))
                return
            for lj in xrange(li, len(myList)):
                one[ni] = myList[lj]
                next_c(lj + 1, ni + 1)
        next_c()
        return answers
    
    
    def findMinimum(self, items, elem):
        """功能: 根据items字典找出elem列表中各项计数的最小值"""
        minimum = items[elem[0]]
        for a in elem:
            if items[a] < minimum:              #如果某元素的计数更小，则记录它的计数
                minimum = items[a]
        return minimum
    
    
    def getL(self, items, min_sup=-1, A=[]):
        """功能: 对于只含单路径的树,进行生成频繁项集"""
        tempResult = []
        finnalResult = []
        nodes = items.keys()        #取得items字典的键，即单路径上的所有节点
        for i in range(1,len(nodes)+1):         #对nodes，即路径上的所有节点生成各种组合
            tempResult += self.combiner(myList=nodes, n=i)
        for elem in tempResult[::-1]:           #elem逆序对dearResult访问，因为接下来会删除元素，逆序好操作
            elemMinimum = self.findMinimum(items, elem)         #找出elem里面的最小计数
            if elemMinimum < min_sup:               #如果组合elem的最小计数小于最小支持度计数，则删除.
                tempResult.remove(elem)
            else:                           #否则把它压入结果列表中进行输出，但它只是条件模式基，要加上最后一个项构成频繁项集，同时把最小计数也加上
                for Aelem in A:         #A可能含有多项
                    if Aelem:
                        temp = elem
                        temp += Aelem
                        temp.append(elemMinimum)
                        finnalResult.append(temp)               #将挖掘出的频繁项集保存在finnalResult列表
        return finnalResult

    


In [2]:
routines = [    
           ['Cola','Egg','Ham'],
           ['Cola','Diaper','Beer'],
           ['Cola','Beer','Diaper','Ham'],
           ['Diaper','Beer']
        ]                                 #事务数据集
min_sup = 3                             #最小支持度计数
headerTable = {}        #头结点表，用来存放各个项的索引

treeBuilder = Tree_builder(routines=routines, min_sup=min_sup, headerTable=headerTable)    #建造FP_Tree
Tree_miner(Tree=treeBuilder.tree, min_sup=min_sup, headerTable=headerTable)         #对FP_Tree进行频繁项集的挖掘

('Diaper', 'Beer') : 3
